In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from skimage.transform import resize  # type: ignore    # %pip install scikit-image
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torchmetrics.classification import BinaryAccuracy  # type: ignore  # %pip install torchmetrics

In [ ]:
# If fname doesn't exists run 'LSWMD-explore.ipynb' to generate it.
fname = "data/LSWMD_onlyLabelData.pkl"
df = pd.read_pickle(fname)

In [ ]:
# Not all wafer maps has equal size. So, explore the images shape.
imgShapes = df["waferMap"].apply(lambda x: x.shape)

# Compute min, max and avg for image shape.
avgShape = tuple(sum(col) / len(col) for col in zip(*imgShapes))
min_max_shape = min(max(imgShapes))
print(
    f"minImgShape={min(imgShapes)}, maxImgShape={max(imgShapes)}, avgImgShape={avgShape}, maxDim={min_max_shape}"
)

minImgShape=(15, 3), maxImgShape=(212, 84), avgImgShape=(35.23280716970223, 34.82212778259613), maxDim=84


In [ ]:
tot = df.shape[0]
good_samples = df[df["failureType"] == "none"].shape[0]
bad_samples = tot - good_samples

print(f"tot = {tot}, good = {good_samples}, bad = {bad_samples}")

tot = 172950, good = 147431, bad = 25519


In [ ]:
# Map non-faulty samples to 0, faulty to 1

df['failureNum'] = np.where(df['failureType'] == 'none', 0, 1)

In [ ]:
# 85% of the data has no fault, but 15% does.
#   So, some synthetic fault data would be generated by applying 2 random rotations to faulty examples.

# TODO:

In [ ]:
# Extract data.
X = df["waferMap"].to_numpy()
Y = df["failureNum"].to_numpy()

# Resize to nsize-by-nsize using nearest-neighbor.
nsize = 36  # 42.86% of max size.
resizef = lambda x: resize(
    x, (nsize, nsize), order=0, preserve_range=True, anti_aliasing=False
).astype(np.uint8)
X = np.array([resizef(x) for x in X])

# NOTE: Size increase 1.3836 mb -> 224.1432 mb

In [ ]:
orgNumBytes = df["waferMap"].to_numpy().nbytes
newNumBytes = X.nbytes

print(orgNumBytes)
print(newNumBytes)

1383600
896572800


In [ ]:
# Define a Convolutional Neural Network.

class Net(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)       # Output: (N, 16, 34, 34)
        self.pool = nn.MaxPool2d(2, 2)         # After pool: (N, 16, 17, 17)
        self.conv2 = nn.Conv2d(16, 16, 3)      # Output: (N, 16, 15, 15)
        # After pool: (N, 16, 7, 7)
        self.fc1 = nn.Linear(16 * 7 * 7, 128)   # (784, 120)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()

In [50]:
# Move the net to GPU, if available.
device = "cuda" if torch.cuda.is_available() else "cpu"
net = net.to(device)

In [72]:
# Define a Loss function and optimizer.

loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Split data into training and testing.

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [55]:
# Convert numpy data to pytorch tensors.

train_tensor_data = torch.from_numpy(X_train).unsqueeze(1).float()  # Add channel of 1.
train_tensor_labels = torch.from_numpy(y_train).long()

test_tensor_data = torch.from_numpy(X_test).unsqueeze(1).float()  # Add channel of 1.
test_tensor_labels = torch.from_numpy(y_test).long()

In [56]:
# Load tensor data.

train_dataset = TensorDataset(train_tensor_data, train_tensor_labels)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = TensorDataset(test_tensor_data, test_tensor_labels)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [73]:
num_epochs = 15

for epoch in range(num_epochs):
    for batch_data, batch_labels in train_dataloader:
        # Zero gradients.
        optimizer.zero_grad()

        # Move data to gpu, if available.
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)

        # forward + backward + optimize.
        outputs = net(batch_data)
        loss_out = loss(outputs, batch_labels)
        loss_out.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss_out.item():.4f}')

print('Finished Training')

Epoch [1/15], Loss: 0.0507
Epoch [2/15], Loss: 0.1267
Epoch [3/15], Loss: 0.0857
Epoch [4/15], Loss: 0.1338
Epoch [5/15], Loss: 0.0318
Epoch [6/15], Loss: 0.0542
Epoch [7/15], Loss: 0.0421
Epoch [8/15], Loss: 0.0309
Epoch [9/15], Loss: 0.0229
Epoch [10/15], Loss: 0.0262
Epoch [11/15], Loss: 0.0199
Epoch [12/15], Loss: 0.0496
Epoch [13/15], Loss: 0.1828
Epoch [14/15], Loss: 0.1235
Epoch [15/15], Loss: 0.1132
Finished Training


In [74]:
# Evaluate model.

def evaluate(
    model: nn.Module,
    criterion: nn.Module,
    minibatcher: DataLoader,
    device: str,
) -> float:
    R"""
    Evaluate.
    """
    model.eval()

    buf_total = []
    buf_metric = []
    for inputs, targets in minibatcher:
        inputs = inputs.to(device)
        targets = targets.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            total = len(targets)
            metric = criterion.forward(outputs, targets).item()
        buf_total.append(total)
        buf_metric.append(metric * total)

    return float(sum(buf_metric)) / float(sum(buf_total))


class Accuracy(nn.Module):
    R"""
    Accuracy module.
    """

    def forward(
        self,
        output: torch.Tensor,
        target: torch.Tensor,
        /,
    ) -> torch.Tensor:
        R"""
        Forward.
        """
        return torch.sum(torch.argmax(output, dim=1) == target) / len(target)

In [75]:
metric = Accuracy()

ce_train = evaluate(net, loss, train_dataloader, device=device)
acc_train = evaluate(net, metric, train_dataloader, device=device)
acc_test = evaluate(net, metric, test_dataloader, device=device)

print("Train Loss: {:.6f}".format(ce_train))
print(" Train Acc: {:.6f}".format(acc_train))
print("  Test Acc: {:.6f}".format(acc_test))


Train Loss: 0.063256
 Train Acc: 0.978910
  Test Acc: 0.969500
